In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.
sample_sets = "AG1000G-BF-A"
contigs = ["3L"]
cohorts_analysis = "20240924"
min_cohort_size = 20
max_cohort_size = 50
dask_scheduler = "single-threaded"
analysis_version = "dev"

# Finalise cohorts

Here we finalise the list of cohorts to include in the site, removing cohorts which fail H12 window size calibration.

In [ ]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/scripts/setup.py

In [ ]:
df_cohorts = pd.read_csv(cohorts_file)
df_cohorts.head()

In [ ]:
window_size = []

for cohort_id in df_cohorts.loc[:, "cohort_id"]:
    # Load window sizes.
    h12_calibration_file = h12_calibration_files.as_posix().format(cohort=cohort_id)
    with open(h12_calibration_file) as params_file:
        window_size_params = yaml.safe_load(params_file)

    window_size.append(window_size_params["h12_window_size"])

df_cohorts.loc[:, "h12_window_size"] = window_size

In [ ]:
df_cohorts = df_cohorts.query("~h12_window_size.isna()", engine="python")
df_cohorts.to_csv(final_cohorts_file, index=None)